# Lab 2 - Linear and Logistic Regression

**Introduction to Machine Learning and Deep Learning** – ELTE Faculty of Informatics, 2024/25 Spring Semester

This notebook demonstrates the material from `prez2a.pdf` (Linear Regression) and `prez2b.pdf` (Logistic Regression, Classification) with code and visualizations.

---

**Contents:**
1. [Linear Regression](#1-linear-regression)
   - 1.1 Simple linear regression from scratch
   - 1.2 Multiple linear regression
   - 1.3 Polynomial regression
   - 1.4 Regularization (Ridge, Lasso, Elastic Net)
   - 1.5 Learning curves
2. [Logistic Regression and Classification](#2-logistic-regression-and-classification)
   - 2.1 Binary logistic regression
   - 2.2 Evaluation metrics
   - 2.3 ROC and Precision-Recall curves
   - 2.4 Multi-class classification
   - 2.5 Imbalanced datasets
3. [Exercises](#3-exercises)

## Imports and Settings

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import (
    mean_squared_error, r2_score,
    accuracy_score, balanced_accuracy_score,
    precision_score, recall_score, f1_score,
    confusion_matrix, classification_report,
    roc_curve, auc, precision_recall_curve, RocCurveDisplay, log_loss
)
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

RANDOM_STATE = 42
print('Imports successful!')

Imports successful!


---
# 1. Linear Regression

The goal of linear regression: predict a continuous target variable ($y$) as a linear combination of input features ($X$).

$$\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \ldots + \theta_n x_n = \mathbf{\theta}^T \mathbf{x}$$

## 1.1 Simple Linear Regression from Scratch

First, we demonstrate the method on synthetic data with a manual implementation.

### Generating Synthetic Data

In [ ]:
np.random.seed(RANDOM_STATE)

# y = 3 + 2*x + noise
X_simple = 2 * np.random.rand(100, 1)
y_simple = 3 + 2 * X_simple + np.random.randn(100, 1) * 0.5

plt.scatter(X_simple, y_simple, alpha=0.6, edgecolors='k', s=40)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Synthetic data: y = 3 + 2x + noise')
plt.show()

: 

### MSE Cost Function

$$J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})^2 = \frac{1}{2m} \sum_{i=1}^{m} (\theta^T x^{(i)} - y^{(i)})^2$$

In [ ]:
def compute_cost(X, y, theta):
    """MSE cost function."""
    m = len(y)
    predictions = X @ theta
    cost = (1 / (2 * m)) * np.sum((predictions - y) ** 2)
    return cost

# Add bias column (x0 = 1)
X_b = np.c_[np.ones((100, 1)), X_simple]

# Test with random theta
theta_init = np.random.randn(2, 1)
print(f'Initial theta: {theta_init.ravel()}')
print(f'Initial cost: {compute_cost(X_b, y_simple, theta_init):.4f}')

: 

### Batch Gradient Descent

The gradient:
$$\frac{\partial J}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)}) \cdot x_j^{(i)}$$

Update rule:
$$\theta_j := \theta_j - \alpha \frac{\partial J}{\partial \theta_j}$$

In [ ]:
def batch_gradient_descent(X, y, theta, learning_rate, n_iterations):
    """Batch gradient descent with cost history."""
    m = len(y)
    cost_history = []
    theta_history = [theta.copy()]

    for i in range(n_iterations):
        gradients = (1 / m) * X.T @ (X @ theta - y)
        theta = theta - learning_rate * gradients
        cost_history.append(compute_cost(X, y, theta))
        theta_history.append(theta.copy())

    return theta, cost_history, theta_history

# Run gradient descent
learning_rate = 0.1
n_iterations = 100
theta_init = np.random.randn(2, 1)

theta_gd, cost_history, theta_history = batch_gradient_descent(
    X_b, y_simple, theta_init, learning_rate, n_iterations
)

print(f'Gradient Descent result: theta0 = {theta_gd[0,0]:.4f}, theta1 = {theta_gd[1,0]:.4f}')
print(f'(True values: theta0 = 3, theta1 = 2)')

: 

In [ ]:
# Visualize cost convergence
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Cost over iterations
axes[0].plot(cost_history, 'b-', linewidth=2)
axes[0].set_xlabel('Iteration')
axes[0].set_ylabel('Cost (MSE/2)')
axes[0].set_title('Cost Function Convergence')

# Regression line at different iterations
axes[1].scatter(X_simple, y_simple, alpha=0.5, s=30, label='Data points')
X_plot = np.array([[0], [2]])
X_plot_b = np.c_[np.ones((2, 1)), X_plot]

for step in [0, 1, 2, 5, 10, n_iterations]:
    theta_step = theta_history[min(step, len(theta_history)-1)]
    y_plot = X_plot_b @ theta_step
    style = 'r-' if step == n_iterations else '--'
    alpha = 1.0 if step == n_iterations else 0.4
    axes[1].plot(X_plot, y_plot, style, alpha=alpha, label=f'Iteration {step}')

axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_title('Regression Line Evolution')
axes[1].legend(fontsize=8)
plt.tight_layout()
plt.show()

: 

### Effect of Learning Rate ($\alpha$)

Too small: slow convergence. Too large: may diverge.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 4))
learning_rates = [0.01, 0.1, 0.5]
titles = ['Too small (\u03b1=0.01)', 'Appropriate (\u03b1=0.1)', 'Large (\u03b1=0.5)']

for ax, lr, title in zip(axes, learning_rates, titles):
    theta_init = np.random.RandomState(42).randn(2, 1)
    _, costs, _ = batch_gradient_descent(X_b, y_simple, theta_init, lr, 50)
    ax.plot(costs, linewidth=2)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Cost')
    ax.set_title(title)
    ax.set_ylim(0, max(costs[0] * 1.1, 5))

plt.tight_layout()
plt.show()

: 

### Stochastic Gradient Descent (SGD)

SGD uses only **one** random sample per iteration to estimate the gradient. Faster but noisier convergence.

In [ ]:
def stochastic_gradient_descent(X, y, theta, learning_rate, n_epochs):
    """Stochastic gradient descent."""
    m = len(y)
    cost_history = []

    for epoch in range(n_epochs):
        indices = np.random.permutation(m)
        for i in indices:
            xi = X[i:i+1]
            yi = y[i:i+1]
            gradient = xi.T @ (xi @ theta - yi)
            theta = theta - learning_rate * gradient
        cost_history.append(compute_cost(X, y, theta))

    return theta, cost_history

theta_init = np.random.RandomState(42).randn(2, 1)
theta_sgd, cost_sgd = stochastic_gradient_descent(X_b, y_simple, theta_init, 0.01, 50)

# Compare BGD vs SGD
theta_init = np.random.RandomState(42).randn(2, 1)
_, cost_bgd, _ = batch_gradient_descent(X_b, y_simple, theta_init, 0.1, 50)

plt.plot(cost_bgd, label='Batch GD', linewidth=2)
plt.plot(cost_sgd, label='Stochastic GD', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.title('Batch GD vs. Stochastic GD')
plt.legend()
plt.show()

print(f'SGD result: theta0 = {theta_sgd[0,0]:.4f}, theta1 = {theta_sgd[1,0]:.4f}')

: 

### Normal Equation

Closed-form solution, no iteration required:

$$\hat{\theta} = (X^T X)^{-1} X^T y$$

In [ ]:
def normal_equation(X, y):
    """Closed-form solution for linear regression."""
    return np.linalg.pinv(X.T @ X) @ X.T @ y

theta_ne = normal_equation(X_b, y_simple)
print(f'Normal equation: theta0 = {theta_ne[0,0]:.4f}, theta1 = {theta_ne[1,0]:.4f}')

: 

### Comparison with sklearn

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_simple, y_simple)

print(f'sklearn:      intercept = {lin_reg.intercept_[0]:.4f}, coef = {lin_reg.coef_[0][0]:.4f}')
print(f'Normal eq:    theta0 = {theta_ne[0,0]:.4f}, theta1 = {theta_ne[1,0]:.4f}')
print(f'Grad. desc:   theta0 = {theta_gd[0,0]:.4f}, theta1 = {theta_gd[1,0]:.4f}')
print('\nAll three methods produce nearly identical results!')

: 

---
## 1.2 Multiple Linear Regression

Demonstrated on real data: **USA Housing** dataset.

In [ ]:
df_housing = pd.read_csv('data/USA_Housing.csv')
print(f'Dataset shape: {df_housing.shape}')
df_housing.head()

: 

In [ ]:
df_housing.describe()

: 

In [ ]:
# Feature matrix and target
feature_cols = ['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
                'Avg. Area Number of Bedrooms', 'Area Population']
X_housing = df_housing[feature_cols]
y_housing = df_housing['Price']

# Train-test split
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(
    X_housing, y_housing, test_size=0.2, random_state=RANDOM_STATE
)

: 

### Feature Scaling

**Standardization** (Z-score): $x' = \frac{x - \mu}{\sigma}$ → mean=0, std=1

**Normalization** (Min-Max): $x' = \frac{x - x_{min}}{x_{max} - x_{min}}$ → [0, 1] range

Important: the scaler must be **fit only on the training data**!

In [ ]:
scaler = StandardScaler()
X_train_h_scaled = scaler.fit_transform(X_train_h)
X_test_h_scaled = scaler.transform(X_test_h)

# Show the effect of scaling
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].boxplot(X_train_h.values, labels=[c[:10] for c in feature_cols])
axes[0].set_title('Before Scaling')
axes[0].tick_params(axis='x', rotation=30)

axes[1].boxplot(X_train_h_scaled, labels=[c[:10] for c in feature_cols])
axes[1].set_title('After Standardization')
axes[1].tick_params(axis='x', rotation=30)

plt.tight_layout()
plt.show()

: 

In [ ]:
# Fit linear regression
lr_housing = LinearRegression()
lr_housing.fit(X_train_h_scaled, y_train_h)

y_pred_h = lr_housing.predict(X_test_h_scaled)

print(f'MSE:  {mean_squared_error(y_test_h, y_pred_h):,.2f}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test_h, y_pred_h)):,.2f}')
print(f'R\u00b2:   {r2_score(y_test_h, y_pred_h):.4f}')

# Coefficients
coef_df = pd.DataFrame({'Feature': feature_cols, 'Coefficient': lr_housing.coef_})
coef_df = coef_df.sort_values('Coefficient', ascending=True)
print(f'\nIntercept: {lr_housing.intercept_:,.2f}')
print('\nCoefficients:')
print(coef_df.to_string(index=False))

: 

In [ ]:
# Residual analysis
residuals = y_test_h - y_pred_h

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Predicted vs actual
axes[0].scatter(y_test_h, y_pred_h, alpha=0.3, s=10)
axes[0].plot([y_test_h.min(), y_test_h.max()], [y_test_h.min(), y_test_h.max()], 'r--', linewidth=2)
axes[0].set_xlabel('Actual Price')
axes[0].set_ylabel('Predicted Price')
axes[0].set_title('Prediction vs. Actual')

# Residual distribution
axes[1].hist(residuals, bins=30, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Residuals')
axes[1].set_title('Residual Distribution')

# Residuals vs predicted
axes[2].scatter(y_pred_h, residuals, alpha=0.3, s=10)
axes[2].axhline(y=0, color='r', linestyle='--')
axes[2].set_xlabel('Predicted Price')
axes[2].set_ylabel('Residuals')
axes[2].set_title('Residuals vs. Prediction')

plt.tight_layout()
plt.show()

: 

---
## 1.3 Polynomial Regression

We can model nonlinear relationships with linear regression by expanding the features using polynomial transformation.

E.g. degree 2: $\hat{y} = \theta_0 + \theta_1 x + \theta_2 x^2$

**Warning:** high degree polynomials can lead to overfitting!

In [ ]:
# Generate nonlinear data
np.random.seed(RANDOM_STATE)
X_poly = np.sort(6 * np.random.rand(80, 1) - 3, axis=0)
y_poly = 0.5 * X_poly**3 - X_poly**2 + 2 + np.random.randn(80, 1) * 3

plt.scatter(X_poly, y_poly, alpha=0.6, edgecolors='k', s=40)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Nonlinear Synthetic Data')
plt.show()

: 

In [ ]:
# Compare different polynomial degrees
degrees = [1, 3, 10, 20]
X_plot_poly = np.linspace(-3, 3, 200).reshape(-1, 1)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
titles = ['Underfitting (degree=1)', 'Good fit (degree=3)', 'Beginning to overfit (degree=10)', 'Overfitting (degree=20)']

for ax, degree, title in zip(axes.ravel(), degrees, titles):
    poly = PolynomialFeatures(degree=degree)
    X_poly_feat = poly.fit_transform(X_poly)
    X_plot_feat = poly.transform(X_plot_poly)

    lr = LinearRegression()
    lr.fit(X_poly_feat, y_poly)
    y_plot = lr.predict(X_plot_feat)

    train_mse = mean_squared_error(y_poly, lr.predict(X_poly_feat))

    ax.scatter(X_poly, y_poly, alpha=0.5, s=30)
    ax.plot(X_plot_poly, y_plot, 'r-', linewidth=2)
    ax.set_title(f'{title}\nTrain MSE: {train_mse:.2f}')
    ax.set_ylim(y_poly.min() - 10, y_poly.max() + 10)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

plt.tight_layout()
plt.show()

: 

### Bias-Variance Tradeoff

- **Underfitting (high bias):** The model is too simple, unable to capture the patterns in the data.
- **Overfitting (high variance):** The model is too complex, it learns the noise too.
- **Goal:** find the right complexity ("sweet spot").

In [ ]:
# Train vs test error for different polynomial degrees
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(
    X_poly, y_poly, test_size=0.3, random_state=RANDOM_STATE
)

degrees_range = range(1, 16)
train_errors = []
test_errors = []

for d in degrees_range:
    poly = PolynomialFeatures(degree=d)
    X_tr = poly.fit_transform(X_train_p)
    X_te = poly.transform(X_test_p)

    lr = LinearRegression()
    lr.fit(X_tr, y_train_p)

    train_errors.append(mean_squared_error(y_train_p, lr.predict(X_tr)))
    test_errors.append(mean_squared_error(y_test_p, lr.predict(X_te)))

plt.plot(list(degrees_range), train_errors, 'b-o', label='Train MSE')
plt.plot(list(degrees_range), test_errors, 'r-o', label='Test MSE')
plt.xlabel('Polynomial Degree')
plt.ylabel('MSE')
plt.title('Bias-Variance Tradeoff: Train vs. Test Error')
plt.legend()
plt.yscale('log')
plt.show()

: 

---
## 1.4 Regularization

Regularization protects against overfitting by adding a penalty term to the cost function.

| Method | Penalty Term | Effect |
|--------|-------------|--------|
| **Ridge (L2)** | $\alpha \sum \theta_j^2$ | Shrinks coefficients toward zero |
| **Lasso (L1)** | $\alpha \sum |\theta_j|$ | Can push coefficients exactly to zero (feature selection) |
| **Elastic Net** | $\alpha (r \sum |\theta_j| + \frac{1-r}{2} \sum \theta_j^2)$ | Combination of L1 and L2 |

In [ ]:
# Demonstrate regularization on high-degree polynomial
degree = 10
poly = PolynomialFeatures(degree=degree)
X_poly_reg = poly.fit_transform(X_poly)
X_plot_reg = poly.transform(X_plot_poly)

# Scale features for regularization
scaler_poly = StandardScaler()
X_poly_reg_s = scaler_poly.fit_transform(X_poly_reg)
X_plot_reg_s = scaler_poly.transform(X_plot_reg)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
models = [
    ('Ridge (L2)', Ridge(alpha=1.0)),
    ('Lasso (L1)', Lasso(alpha=0.1)),
    ('Elastic Net', ElasticNet(alpha=0.1, l1_ratio=0.5))
]

for ax, (name, model) in zip(axes, models):
    model.fit(X_poly_reg_s, y_poly.ravel())
    y_plot = model.predict(X_plot_reg_s)

    ax.scatter(X_poly, y_poly, alpha=0.5, s=30)
    ax.plot(X_plot_poly, y_plot, 'r-', linewidth=2)
    ax.set_title(f'{name}\nNon-zero coefficients: {np.sum(np.abs(model.coef_) > 1e-6)}/{len(model.coef_)}')
    ax.set_ylim(y_poly.min() - 10, y_poly.max() + 10)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

plt.tight_layout()
plt.show()

: 

In [ ]:
# Alpha effect on Ridge coefficients
alphas = np.logspace(-2, 4, 100)
coefs_ridge = []

for a in alphas:
    ridge = Ridge(alpha=a)
    ridge.fit(X_poly_reg_s, y_poly.ravel())
    coefs_ridge.append(ridge.coef_)

coefs_ridge = np.array(coefs_ridge)

plt.figure(figsize=(10, 6))
for i in range(coefs_ridge.shape[1]):
    plt.plot(alphas, coefs_ridge[:, i], linewidth=1)

plt.xscale('log')
plt.xlabel('Alpha (regularization parameter)')
plt.ylabel('Coefficient value')
plt.title('Ridge: Coefficient paths as a function of alpha')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.show()

: 

---
## 1.5 Learning Curves

The learning curve shows how the training and validation error change as the amount of training data increases.

- **High bias:** both curves converge at a high error level (→ need a more complex model)
- **High variance:** large gap between training and validation error (→ need more data or regularization)

In [ ]:
def plot_learning_curve(estimator, X, y, title, ax):
    """Plot learning curve for a given estimator."""
    train_sizes, train_scores, val_scores = learning_curve(
        estimator, X, y, cv=5,
        train_sizes=np.linspace(0.1, 1.0, 10),
        scoring='neg_mean_squared_error',
        random_state=RANDOM_STATE
    )
    train_mean = -train_scores.mean(axis=1)
    train_std = train_scores.std(axis=1)
    val_mean = -val_scores.mean(axis=1)
    val_std = val_scores.std(axis=1)

    ax.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
    ax.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
    ax.plot(train_sizes, train_mean, 'b-o', label='Train', markersize=4)
    ax.plot(train_sizes, val_mean, 'r-o', label='Validation', markersize=4)
    ax.set_xlabel('Number of Training Samples')
    ax.set_ylabel('MSE')
    ax.set_title(title)
    ax.legend()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Underfitting model
pipe_linear = Pipeline([
    ('poly', PolynomialFeatures(degree=1)),
    ('lr', LinearRegression())
])
plot_learning_curve(pipe_linear, X_poly, y_poly.ravel(), 'Linear (high bias)', axes[0])

# Good model
pipe_cubic = Pipeline([
    ('poly', PolynomialFeatures(degree=3)),
    ('lr', LinearRegression())
])
plot_learning_curve(pipe_cubic, X_poly, y_poly.ravel(), 'Cubic (good fit)', axes[1])

# Overfitting model
pipe_high = Pipeline([
    ('poly', PolynomialFeatures(degree=15)),
    ('lr', LinearRegression())
])
plot_learning_curve(pipe_high, X_poly, y_poly.ravel(), 'Degree 15 (high variance)', axes[2])

plt.tight_layout()
plt.show()

: 

---
# 2. Logistic Regression and Classification

Logistic regression is used for **classification** tasks. The output is a probability (between $0$ and $1$), ensured by the **sigmoid** function:

$$\sigma(z) = \frac{1}{1 + e^{-z}}, \quad \text{where } z = \theta^T x$$

## 2.1 Binary Logistic Regression

### The Sigmoid Function

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

z = np.linspace(-7, 7, 200)

plt.figure(figsize=(8, 5))
plt.plot(z, sigmoid(z), 'b-', linewidth=2.5)
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='Threshold = 0.5')
plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
plt.axhline(y=1, color='gray', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
plt.xlabel('z = \u03b8\u1d40x')
plt.ylabel('\u03c3(z)')
plt.title('Sigmoid Function')
plt.legend()
plt.ylim(-0.05, 1.05)
plt.show()

: 

### Decision Boundary

In [ ]:
# Generate 2D classification data
np.random.seed(RANDOM_STATE)
X_cls, y_cls = make_classification(
    n_samples=200, n_features=2, n_redundant=0, n_informative=2,
    n_clusters_per_class=1, random_state=RANDOM_STATE
)

# Fit logistic regression
log_reg = LogisticRegression(random_state=RANDOM_STATE)
log_reg.fit(X_cls, y_cls)

# Plot decision boundary
def plot_decision_boundary(model, X, y, ax=None, title=''):
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.3, cmap='RdBu')
    ax.scatter(X[y==0, 0], X[y==0, 1], c='blue', edgecolors='k', s=40, label='Class 0')
    ax.scatter(X[y==1, 0], X[y==1, 1], c='red', edgecolors='k', s=40, label='Class 1')
    ax.set_xlabel('x\u2081')
    ax.set_ylabel('x\u2082')
    ax.set_title(title)
    ax.legend()

plot_decision_boundary(log_reg, X_cls, y_cls, title='Logistic Regression Decision Boundary')
plt.show()

print(f'Accuracy: {log_reg.score(X_cls, y_cls):.4f}')

: 

### Cost Function (Cross-Entropy / Log-Loss)

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log(\hat{p}^{(i)}) + (1-y^{(i)}) \log(1 - \hat{p}^{(i)}) \right]$$

In [ ]:
# Visualize log-loss for a single sample
p = np.linspace(0.001, 0.999, 200)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# When y = 1
axes[0].plot(p, -np.log(p), 'b-', linewidth=2)
axes[0].set_xlabel('Predicted probability p')
axes[0].set_ylabel('Loss')
axes[0].set_title('Cost when y = 1: -log(p)')

# When y = 0
axes[1].plot(p, -np.log(1 - p), 'r-', linewidth=2)
axes[1].set_xlabel('Predicted probability p')
axes[1].set_ylabel('Loss')
axes[1].set_title('Cost when y = 0: -log(1-p)')

plt.tight_layout()
plt.show()
print('If the prediction is correct (p close to y) -> low cost')
print('If the prediction is wrong (p far from y)  -> very high cost')

: 

---
## 2.2 Evaluation Metrics

Demonstrated on real data: **Heart Disease** dataset.

In [ ]:
df_heart = pd.read_csv('data/heart-disease.csv')
print(f'Dataset shape: {df_heart.shape}')
print(f'\nTarget distribution:')
print(df_heart['target'].value_counts())
df_heart.head()

: 

In [ ]:
# Prepare data
X_heart = df_heart.drop('target', axis=1)
y_heart = df_heart['target']

X_train_heart, X_test_heart, y_train_heart, y_test_heart = train_test_split(
    X_heart, y_heart, test_size=0.2, random_state=RANDOM_STATE, stratify=y_heart
)

# Scale features
scaler_heart = StandardScaler()
X_train_heart_s = scaler_heart.fit_transform(X_train_heart)
X_test_heart_s = scaler_heart.transform(X_test_heart)

# Fit logistic regression
log_reg_heart = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
log_reg_heart.fit(X_train_heart_s, y_train_heart)

y_pred_heart = log_reg_heart.predict(X_test_heart_s)
y_prob_heart = log_reg_heart.predict_proba(X_test_heart_s)[:, 1]

: 

### Confusion Matrix

|  | Predicted: Positive | Predicted: Negative |
|---|---|---|
| **Actual: Positive** | TP (True Positive) | FN (False Negative) |
| **Actual: Negative** | FP (False Positive) | TN (True Negative) |

In [ ]:
cm = confusion_matrix(y_test_heart, y_pred_heart)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No disease (0)', 'Disease (1)'],
            yticklabels=['No disease (0)', 'Disease (1)'])
plt.xlabel('Prediction')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

: 

### Precision, Recall, F1-score

- **Precision**: $\frac{TP}{TP + FP}$ – Of all predicted positives, how many are truly positive?
- **Recall** (Sensitivity): $\frac{TP}{TP + FN}$ – Of all actual positives, how many did we find?
- **F1-score**: $2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$ – Harmonic mean

In [ ]:
print('Classification Report:')
print('=' * 55)
print(classification_report(y_test_heart, y_pred_heart,
                            target_names=['No disease', 'Disease']))

print(f'Accuracy:          {accuracy_score(y_test_heart, y_pred_heart):.4f}')
print(f'Balanced accuracy: {balanced_accuracy_score(y_test_heart, y_pred_heart):.4f}')

: 

---
## 2.3 ROC and Precision-Recall Curves

### ROC Curve (Receiver Operating Characteristic)

The ROC curve shows the relationship between **True Positive Rate** (Recall) and **False Positive Rate** at different thresholds.

- **AUC = 1.0**: perfect classifier
- **AUC = 0.5**: random (diagonal line)

In [ ]:
fpr, tpr, thresholds_roc = roc_curve(y_test_heart, y_prob_heart)
roc_auc = auc(fpr, tpr)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC curve
axes[0].plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC (AUC = {roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], 'r--', alpha=0.5, label='Random')
axes[0].fill_between(fpr, tpr, alpha=0.1)
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate (Recall)')
axes[0].set_title('ROC Curve')
axes[0].legend()

# Precision-Recall curve
precision_curve, recall_curve, thresholds_pr = precision_recall_curve(y_test_heart, y_prob_heart)
axes[1].plot(recall_curve, precision_curve, 'g-', linewidth=2)
axes[1].fill_between(recall_curve, precision_curve, alpha=0.1, color='green')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')

plt.tight_layout()
plt.show()

: 

### Threshold Tuning

The default threshold is 0.5, but in medical diagnostics a high recall (few FN) may be more important, while in spam filtering a high precision (few FP) is preferred.

In [ ]:
thresholds = np.arange(0.1, 0.95, 0.05)
results = []

for t in thresholds:
    y_pred_t = (y_prob_heart >= t).astype(int)
    results.append({
        'Threshold': t,
        'Precision': precision_score(y_test_heart, y_pred_t, zero_division=0),
        'Recall': recall_score(y_test_heart, y_pred_t, zero_division=0),
        'F1': f1_score(y_test_heart, y_pred_t, zero_division=0),
        'Accuracy': accuracy_score(y_test_heart, y_pred_t)
    })

df_thresh = pd.DataFrame(results)

plt.figure(figsize=(10, 6))
plt.plot(df_thresh['Threshold'], df_thresh['Precision'], 'b-o', label='Precision', markersize=4)
plt.plot(df_thresh['Threshold'], df_thresh['Recall'], 'r-o', label='Recall', markersize=4)
plt.plot(df_thresh['Threshold'], df_thresh['F1'], 'g-o', label='F1', markersize=4)
plt.plot(df_thresh['Threshold'], df_thresh['Accuracy'], 'k--', label='Accuracy', markersize=4, alpha=0.5)
plt.axvline(x=0.5, color='gray', linestyle=':', label='Default threshold')
plt.xlabel('Decision Threshold')
plt.ylabel('Metric Value')
plt.title('Metrics as a Function of Threshold')
plt.legend()
plt.show()

: 

---
## 2.4 Multi-class Classification

Two main approaches:
- **One-vs-Rest (OvR):** $K$ binary models, each distinguishing one class from the rest
- **Softmax Regression (Multinomial):** A single model that directly handles $K$ classes

$$P(y=k | x) = \frac{e^{\theta_k^T x}}{\sum_{j=1}^{K} e^{\theta_j^T x}}$$

In [ ]:
# Load Iris dataset
df_iris = pd.read_csv('data/iris.csv')
print(f'Iris dataset shape: {df_iris.shape}')
print(f'\nClass distribution:')
print(df_iris['target'].value_counts().sort_index())

target_names = ['setosa', 'versicolor', 'virginica']

: 

In [ ]:
X_iris = df_iris.drop('target', axis=1).values
y_iris = df_iris['target'].astype(int).values

X_train_iris, X_test_iris, y_train_iris, y_test_iris = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=RANDOM_STATE, stratify=y_iris
)

scaler_iris = StandardScaler()
X_train_iris_s = scaler_iris.fit_transform(X_train_iris)
X_test_iris_s = scaler_iris.transform(X_test_iris)

# Compare OvR vs Softmax
models_mc = {
    'One-vs-Rest (OvR)': LogisticRegression(multi_class='ovr', max_iter=1000, random_state=RANDOM_STATE),
    'Softmax (Multinomial)': LogisticRegression(multi_class='multinomial', max_iter=1000, random_state=RANDOM_STATE)
}

for name, model in models_mc.items():
    model.fit(X_train_iris_s, y_train_iris)
    y_pred = model.predict(X_test_iris_s)
    acc = accuracy_score(y_test_iris, y_pred)
    print(f'\n{name}:')
    print(f'  Accuracy: {acc:.4f}')
    print(classification_report(y_test_iris, y_pred, target_names=target_names))

: 

In [ ]:
# Multi-class confusion matrix
softmax_model = models_mc['Softmax (Multinomial)']
y_pred_iris = softmax_model.predict(X_test_iris_s)

cm_iris = confusion_matrix(y_test_iris, y_pred_iris)

plt.figure(figsize=(7, 5))
sns.heatmap(cm_iris, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.xlabel('Prediction')
plt.ylabel('Actual')
plt.title('Multi-class Confusion Matrix (Iris - Softmax)')
plt.show()

: 

### Decision Boundary Visualization in 2D (first 2 features)

In [ ]:
# Use only first 2 features for visualization
X_iris_2d = X_iris[:, :2]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, (name, mc_type) in zip(axes, [('OvR', 'ovr'), ('Softmax', 'multinomial')]):
    model = LogisticRegression(multi_class=mc_type, max_iter=1000, random_state=RANDOM_STATE)
    model.fit(X_iris_2d, y_iris)

    x_min, x_max = X_iris_2d[:, 0].min() - 0.5, X_iris_2d[:, 0].max() + 0.5
    y_min, y_max = X_iris_2d[:, 1].min() - 0.5, X_iris_2d[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    ax.contourf(xx, yy, Z, alpha=0.3, cmap='Set1')
    for cls, marker, label in zip([0, 1, 2], ['o', 's', '^'], target_names):
        ax.scatter(X_iris_2d[y_iris==cls, 0], X_iris_2d[y_iris==cls, 1],
                   marker=marker, edgecolors='k', s=50, label=label)
    ax.set_xlabel('Sepal length')
    ax.set_ylabel('Sepal width')
    ax.set_title(f'{name} Decision Boundary')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

: 

---
## 2.5 Imbalanced Datasets

When classes have very different sizes, accuracy can be misleading!

Solutions:
- `class_weight='balanced'` (weights the minority class more heavily)
- Sampling techniques (SMOTE, etc.)
- Using appropriate metrics (F1, balanced accuracy, AUC)

In [ ]:
# Create imbalanced dataset
X_imb, y_imb = make_classification(
    n_samples=1000, n_features=10, n_informative=5,
    n_classes=2, weights=[0.9, 0.1],
    random_state=RANDOM_STATE
)

print(f'Class distribution: 0 -> {np.sum(y_imb==0)}, 1 -> {np.sum(y_imb==1)}')
print(f'Ratio: {np.sum(y_imb==0)/len(y_imb):.1%} vs {np.sum(y_imb==1)/len(y_imb):.1%}')

X_train_imb, X_test_imb, y_train_imb, y_test_imb = train_test_split(
    X_imb, y_imb, test_size=0.3, random_state=RANDOM_STATE, stratify=y_imb
)

: 

In [ ]:
# Compare: without vs with class_weight='balanced'
models_imb = {
    'Default (no weighting)': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    'class_weight=\'balanced\'': LogisticRegression(class_weight='balanced', random_state=RANDOM_STATE, max_iter=1000)
}

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, (name, model) in zip(axes, models_imb.items()):
    model.fit(X_train_imb, y_train_imb)
    y_pred = model.predict(X_test_imb)

    cm = confusion_matrix(y_test_imb, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Negative', 'Positive'],
                yticklabels=['Negative', 'Positive'])
    ax.set_xlabel('Prediction')
    ax.set_ylabel('Actual')

    acc = accuracy_score(y_test_imb, y_pred)
    bal_acc = balanced_accuracy_score(y_test_imb, y_pred)
    f1 = f1_score(y_test_imb, y_pred)
    rec = recall_score(y_test_imb, y_pred)
    ax.set_title(f'{name}\nAcc={acc:.3f}  Bal.Acc={bal_acc:.3f}  F1={f1:.3f}  Recall={rec:.3f}')

plt.tight_layout()
plt.show()

print('\nNote: the default model achieves high accuracy but low recall on the minority class!')
print('Balanced weighting improves recall, but accuracy may slightly decrease.')

: 

In [ ]:
# ROC comparison on imbalanced data
fig, ax = plt.subplots(figsize=(8, 6))

for name, model in models_imb.items():
    y_prob = model.predict_proba(X_test_imb)[:, 1]
    fpr_i, tpr_i, _ = roc_curve(y_test_imb, y_prob)
    auc_i = auc(fpr_i, tpr_i)
    ax.plot(fpr_i, tpr_i, linewidth=2, label=f'{name} (AUC={auc_i:.3f})')

ax.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Comparison - Imbalanced Data')
ax.legend()
plt.show()

: 

---
# 3. Exercises

---

## 3.1 In-class Exercises – Linear Regression

### E1. Mini-batch Gradient Descent

Implement the **mini-batch gradient descent** algorithm! Use the synthetic data generated in Section 1.1 (`X_b`, `y_simple`).

**Steps:**
1. Write a `mini_batch_gradient_descent(X, y, theta, learning_rate, n_epochs, batch_size)` function
2. Run it with `batch_size = 16`
3. Plot the cost function convergence and compare it with batch GD on the same graph
4. Try different `batch_size` values (8, 16, 32, 64) and visualize!

In [ ]:
# --- IN-CLASS EXERCISE E1: Mini-batch GD ---
# Write your solution here!



: 

### E2. Linear Regression on Real Data

Use the `data/diabetes.csv` dataset!

**Steps:**
1. Load the data and perform a brief EDA (descriptive statistics, correlation matrix heatmap)
2. Select the target variable (e.g. `BMI` or `Glucose`) and the features
3. Perform train-test split (80-20), then standardize the features
4. Fit a `LinearRegression` model
5. Evaluate: MSE, RMSE, R\u00b2 metrics
6. Create a residual plot

In [ ]:
# --- IN-CLASS EXERCISE E2: Diabetes regression ---
# Write your solution here!



: 

### E3. Finding the Optimal Polynomial Degree

Use the nonlinear data generated in Section 1.3 (`X_poly`, `y_poly`).

**Steps:**
1. Create a `Pipeline`: `PolynomialFeatures(degree=d)` + `LinearRegression()`
2. Use 5-fold cross-validation (`cross_val_score`) to try degrees from 1 to 15
3. Plot the mean CV error (neg_mean_squared_error) as a function of the degree
4. Which degree is the best? Why?

In [ ]:
# --- IN-CLASS EXERCISE E3: Optimal polynomial degree ---
# Write your solution here!



: 

---
## 3.2 In-class Exercises – Logistic Regression

### E4. Heart Disease Classification

Use the `data/heart-disease.csv` dataset!

**Steps:**
1. Load the data and perform a brief EDA (class distribution, correlation matrix)
2. Train-test split (80-20, stratified), standardization
3. Fit a logistic regression model
4. Plot: (a) confusion matrix heatmap, (b) ROC curve with AUC value
5. Print the classification report

In [ ]:
# --- IN-CLASS EXERCISE E4: Heart Disease classification ---
# Write your solution here!



: 

### E5. Threshold Tuning

Use the heart disease model from Section 2.3 (`log_reg_heart`, `y_prob_heart`).

**Steps:**
1. Set the decision threshold to 0.3: `y_pred_03 = (y_prob_heart >= 0.3).astype(int)`
2. Also set the decision threshold to 0.7
3. For all three thresholds (0.3, 0.5, 0.7), compute: precision, recall, F1, accuracy
4. Compare the results in a table (DataFrame)
5. **Question:** In medical diagnostics, which threshold would you choose, and why?

In [ ]:
# --- IN-CLASS EXERCISE E5: Threshold tuning ---
# Write your solution here!



: 

### E6. Multi-class Classification – Iris

Use the Iris dataset!

**Steps:**
1. Fit logistic regression with both OvR and Softmax strategies
2. Plot a confusion matrix for each
3. Compare the results (accuracy, macro F1)
4. **Question:** Is there a significant difference between the two approaches on this data? When would the difference be larger?

In [ ]:
# --- IN-CLASS EXERCISE E6: Multi-class classification ---
# Write your solution here!



: 

---
## 3.3 Homework – Linear Regression

### HW1. USA Housing – Full Regression Analysis

Perform a **complete regression analysis** on the `USA_Housing.csv` data!

**Requirements:**
1. **EDA** (min. 3 visualizations): distributions, correlations, scatter plots
2. **Preprocessing:** train-test split, feature scaling (try both StandardScaler and MinMaxScaler)
3. **Modeling:** Linear Regression, Ridge, Lasso, Elastic Net (min. 3 alpha values for each)
4. **Comparison:** Create a summary table with MSE and R\u00b2 values for all models
5. **Learning curves:** Plot the learning curve for the best model
6. **Conclusion:** Which model performs best and why? Is there overfitting?

In [ ]:
# --- HOMEWORK HW1: USA Housing regression analysis ---
# Write your solution here!



: 

### HW2. Normal Equation vs. Gradient Descent – Comparison

Compare the efficiency of the normal equation and gradient descent on data of different sizes!

**Requirements:**
1. Generate synthetic data of various sizes: $m \in \{100, 500, 1000, 5000, 10000, 50000\}$ and $n \in \{5, 10, 50\}$ features
2. Implement the normal equation and batch gradient descent from scratch
3. Measure the runtime for both methods (use the `time` module)
4. Plot the runtimes as a function of $m$ and $n$ (2 plots)
5. **Question:** When should you use the normal equation, and when GD? (Theoretically: NE \u2208 $O(n^3)$, GD \u2208 $O(m \cdot n \cdot k)$)

In [ ]:
# --- HOMEWORK HW2: Normal equation vs. GD ---
# Write your solution here!



: 

---
## 3.4 Homework – Logistic Regression

### HW3. Titanic – Full Classification Analysis

Perform a **complete classification analysis** on `data/titanic_train.csv`!

**Requirements:**
1. **EDA:** Examine the data (missing values, distributions, relationships with survival)
2. **Feature engineering:**
   - Handle missing values (`Age` -> median, `Embarked` -> mode, `Cabin` -> drop)
   - Encode categorical variables (`Sex`, `Embarked`)
   - Optional: create new features (e.g. `FamilySize = SibSp + Parch + 1`)
3. **Modeling:** Logistic regression (try: C=0.01, 0.1, 1, 10)
4. **Evaluation:** Confusion matrix, classification report, ROC curve
5. **Regularization:** Compare L1 and L2 regularization (penalty='l1' vs 'l2')
6. **Conclusion:** Which features are most important for predicting survival? (analyze coefficients)

In [ ]:
# --- HOMEWORK HW3: Titanic classification ---
# Write your solution here!



: 

### HW4. Imbalanced Data – Metrics Analysis

Examine how different metrics behave on imbalanced data!

**Requirements:**
1. Generate imbalanced data using `make_classification` with different ratios:
   - Mild: 70%-30%
   - Moderate: 90%-10%
   - Severe: 99%-1%
2. For each ratio, fit logistic regression:
   - (a) `class_weight=None` (default)
   - (b) `class_weight='balanced'`
3. For each combination, compute: accuracy, balanced_accuracy, precision, recall, F1, AUC
4. Create a summary table (DataFrame) and visualize the results
5. **Questions:**
   - Which metric is most informative on imbalanced data?
   - Up to what imbalance ratio is accuracy still sufficient?
   - When should you use the `class_weight='balanced'` setting?

In [ ]:
# --- HOMEWORK HW4: Imbalanced metrics analysis ---
# Write your solution here!



: 